In [1]:
import pandas as pd
import numpy as np

<br>

In [37]:
source_data = pd.read_csv("./data_77-07.csv", index_col=0).astype(float)
source_data.index = pd.PeriodIndex(start='1977', end='2018', freq='q')[:-1]

In [38]:
source_data.head()

,Pc_Cpdr,Pc_Cpdr_d,Pc_Cpdr_nd,Pc_Cp,Pc_FBCF,Pc_FBCF_me,Pc_FBCF_t,Pc_FBCF_C,Pc_FBCF_O,Pc_VE,...,Pc_Rd,Pc_P,Pc_Tp,Etc_ET,Etc_ET_Tpo,Etc_ET_ofe,K_PA,K_PA_ET,K_PA_D,Ep_Td
1977Q1,605.0,58.6,546.4,119.4,281.6,55.5,42.2,170.2,13.7,26.4,...,754.4,149.4,19.8,3690.5,3068.4,622.1,4074.2,3877.2,197.1,4.8
1977Q2,637.2,63.4,573.9,126.8,309.7,67.9,46.6,178.9,16.2,29.6,...,788.2,150.9,19.1,3666.0,3054.2,611.8,4093.0,3895.0,198.0,4.8
1977Q3,669.7,64.8,604.9,134.5,315.9,72.7,47.6,178.6,16.9,36.0,...,818.4,148.8,18.2,3658.9,3055.4,603.5,4117.5,3915.0,202.4,4.9
1977Q4,691.3,67.6,623.7,142.8,333.7,82.2,49.5,183.7,18.2,45.7,...,857.9,166.6,19.4,3675.1,3076.8,598.3,4149.4,3942.5,207.0,5.0
1978Q1,728.7,70.1,658.6,151.7,322.4,81.3,49.1,174.1,17.8,58.5,...,915.4,186.7,20.4,3729.8,3125.6,604.3,4185.5,3970.9,214.6,5.1


<br>

In [39]:
data_exp_pc_cols = [v for v in source_data.columns if v[:2]=="Pc"]

In [47]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
 
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    print(n_vars)
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [64]:
tempp = series_to_supervised(source_data[data_exp_pc_cols], n_in=2, n_out=1, dropnan=True)

28


In [65]:
tempp.columns

Index(['var1(t-2)', 'var2(t-2)', 'var3(t-2)', 'var4(t-2)', 'var5(t-2)',
       'var6(t-2)', 'var7(t-2)', 'var8(t-2)', 'var9(t-2)', 'var10(t-2)',
       'var11(t-2)', 'var12(t-2)', 'var13(t-2)', 'var14(t-2)', 'var15(t-2)',
       'var16(t-2)', 'var17(t-2)', 'var18(t-2)', 'var19(t-2)', 'var20(t-2)',
       'var21(t-2)', 'var22(t-2)', 'var23(t-2)', 'var24(t-2)', 'var25(t-2)',
       'var26(t-2)', 'var27(t-2)', 'var28(t-2)', 'var1(t-1)', 'var2(t-1)',
       'var3(t-1)', 'var4(t-1)', 'var5(t-1)', 'var6(t-1)', 'var7(t-1)',
       'var8(t-1)', 'var9(t-1)', 'var10(t-1)', 'var11(t-1)', 'var12(t-1)',
       'var13(t-1)', 'var14(t-1)', 'var15(t-1)', 'var16(t-1)', 'var17(t-1)',
       'var18(t-1)', 'var19(t-1)', 'var20(t-1)', 'var21(t-1)', 'var22(t-1)',
       'var23(t-1)', 'var24(t-1)', 'var25(t-1)', 'var26(t-1)', 'var27(t-1)',
       'var28(t-1)', 'var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)',
       'var6(t)', 'var7(t)', 'var8(t)', 'var9(t)', 'var10(t)', 'var11(t)',
       'var12(t)', '

In [41]:
source_data[data_exp_pc_cols]

,Pc_Cpdr,Pc_Cpdr_d,Pc_Cpdr_nd,Pc_Cp,Pc_FBCF,Pc_FBCF_me,Pc_FBCF_t,Pc_FBCF_C,Pc_FBCF_O,Pc_VE,...,Pc_Rt_Rpt,Pc_Rrp,Pc_Psotc,Pc_Psotc_i,Pc_Psotc_e,Pc_Id,Pc_CS,Pc_Rd,Pc_P,Pc_Tp
1977Q1,605.0,58.6,546.4,119.4,281.6,55.5,42.2,170.2,13.7,26.4,...,0.2,147.6,145.8,99.8,46.0,31.5,86.7,754.4,149.4,19.8
1977Q2,637.2,63.4,573.9,126.8,309.7,67.9,46.6,178.9,16.2,29.6,...,0.2,156.7,158.2,107.0,51.2,32.7,93.5,788.2,150.9,19.1
1977Q3,669.7,64.8,604.9,134.5,315.9,72.7,47.6,178.6,16.9,36.0,...,0.2,173.0,161.8,112.4,49.3,33.7,98.3,818.4,148.8,18.2
1977Q4,691.3,67.6,623.7,142.8,333.7,82.2,49.5,183.7,18.2,45.7,...,0.2,200.3,164.1,115.9,48.2,37.1,101.4,857.9,166.6,19.4
1978Q1,728.7,70.1,658.6,151.7,322.4,81.3,49.1,174.1,17.8,58.5,...,0.2,216.3,178.0,119.3,58.7,37.8,107.6,915.4,186.7,20.4
1978Q2,766.7,73.3,693.4,159.9,338.4,85.7,50.2,184.6,17.9,60.1,...,0.2,238.3,198.2,126.1,72.1,40.0,113.4,980.6,213.8,21.8
1978Q3,812.3,76.9,735.5,168.9,374.1,86.9,51.1,216.6,19.4,50.3,...,0.2,274.2,212.1,130.1,82.1,45.8,118.0,1049.6,237.3,22.6
1978Q4,858.3,80.0,778.3,177.9,400.0,85.4,51.2,243.0,20.4,29.2,...,0.2,294.5,243.2,138.3,104.9,49.2,121.4,1119.0,260.7,23.3
1979Q1,896.8,86.4,810.4,188.0,463.7,87.1,51.9,302.8,21.8,-3.2,...,0.2,319.8,258.9,138.7,120.2,53.4,122.4,1170.9,274.1,23.4
1979Q2,945.0,90.4,854.6,199.5,493.7,93.9,52.9,324.0,22.9,-13.0,...,0.2,336.8,274.3,147.3,127.0,55.1,131.0,1232.5,287.4,23.3
